In [1]:
isColab = False

if(isColab):
    # connect google drive
    from google.colab import drive
    drive.mount('/content/drive')

# Import library

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
if(isColab): from google.colab.patches import cv2_imshow

# Part1: goodFeaturesToTrack
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [3]:
def custom_box_filter(image, kernel_size):
    # Pad the image with zeros to handle border cases
    pad_size = kernel_size // 2
    if len(image.shape) == 3:
        padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), 'constant')
    else:
        padded_image = np.pad(image, pad_size, 'constant')

    # Compute the integral image
    integral_img = np.cumsum(np.cumsum(padded_image, axis=0), axis=1)

    # Compute the sums using the integral image
    sum_top_right = integral_img[kernel_size:, kernel_size:]
    sum_top_left = integral_img[kernel_size:, :-kernel_size]
    sum_bottom_right = integral_img[:-kernel_size, kernel_size:]
    sum_bottom_left = integral_img[:-kernel_size, :-kernel_size]

    sums = sum_top_right - sum_top_left - sum_bottom_right + sum_bottom_left

    # Divide by kernel area to get the average
    filtered_image = sums / (kernel_size ** 2)

    # Crop the filtered image to the original size
    return filtered_image[pad_size:-pad_size, pad_size:-pad_size]

In [4]:
def custom_blur(image, kernel_size):
    if len(image.shape) == 2:  # Grayscale image
        rows, cols = image.shape
        image_blurred = np.zeros_like(image, dtype=np.float32)

        for i in range(rows):
            for j in range(cols):
                start_row, end_row = max(0, i - kernel_size // 2), min(rows, i + kernel_size // 2 + 1)
                start_col, end_col = max(0, j - kernel_size // 2), min(cols, j + kernel_size // 2 + 1)
                image_blurred[i, j] = np.mean(image[start_row:end_row, start_col:end_col])

    elif len(image.shape) == 3:  # RGB image
        rows, cols, channels = image.shape
        image_blurred = np.zeros_like(image, dtype=np.float32)

        for c in range(channels):
            for i in range(rows):
                for j in range(cols):
                    start_row, end_row = max(0, i - kernel_size // 2), min(rows, i + kernel_size // 2 + 1)
                    start_col, end_col = max(0, j - kernel_size // 2), min(cols, j + kernel_size // 2 + 1)
                    image_blurred[i, j, c] = np.mean(image[start_row:end_row, start_col:end_col, c])

    else:
        raise ValueError("Unsupported image shape")

    return image_blurred

In [5]:
import numpy as np

def goodFeaturesToTrack(image, maxCorners=100, qualityLevel=0.03, blocksize=7):
    
    # Image blurring with averaging filter
    image = custom_blur(image, blocksize)
    
    # Compute gradients using Sobel filter
    Ix = np.gradient(image, axis=1)
    Iy = np.gradient(image, axis=0)
    
    # Compute products of gradients at each pixel
    Ixx = Ix**2
    Iyy = Iy**2
    Ixy = Ix*Iy
    
    # Compute the sums of products of gradients in local windows
    Sxx = custom_box_filter(Ixx, blocksize)
    Syy = custom_box_filter(Iyy, blocksize)
    Sxy = custom_box_filter(Ixy, blocksize)

    # Compute the determinant and trace of the matrix M for each pixel
    detM = (Sxx * Syy) - (Sxy**2)
    traceM = Sxx + Syy
    
    # Compute the Harris response with detM and traceM
    harris_response = detM - 0.04 * (traceM**2)
    
    # Threshold the Harris response to find candidate corners
    threshold = qualityLevel * harris_response.max()
    corners = np.argwhere(harris_response > threshold)
    
    # Sort the corners by Harris response in descending order
    sorted_corners = corners[np.argsort(harris_response[corners[:, 0], corners[:, 1]])[::-1]]
    
    # Keep the top 'maxCorners' corners
    selected_corners = sorted_corners[:maxCorners]

    final_corners = np.array(selected_corners)
    final_corners = final_corners.reshape(-1, 1, 2)

    return final_corners


# Part2: Optical flow with Lukas-Kanade
- Fill the missing part (denoted as ```fill here```) of the code
- We provide procedure comments for complete the function

In [6]:

def optical_flow(old_frame, new_frame, window_size, min_quality):

    feature_list = goodFeaturesToTrack(old_frame, max_corners, min_quality, blocksize)

    w = int(window_size/2)

    # Normalize
    old_frame = old_frame / 255.0
    new_frame = new_frame / 255.0

    # Convolve to get gradients w.r.to X, Y and T dimensions
    kernel_x = np.array([[-1, 1], [-1, 1]]) * 0.25  # Sobel kernel for x-gradient
    kernel_y = np.array([[-1, -1], [1, 1]]) * 0.25  # Sobel kernel for y-gradient
    kernel_t = np.array([[1, 1], [1, 1]]) * 0.25  # Simple difference kernel for t-gradient


    # cv2.filter2D is allowed for convolution!
    fx = cv2.filter2D(old_frame, -1, kernel_x)
    fy = cv2.filter2D(old_frame, -1, kernel_y)
    ft = cv2.filter2D(new_frame, -1, kernel_t) - cv2.filter2D(old_frame, -1, kernel_t)

    u = np.zeros(old_frame.shape)
    v = np.zeros(old_frame.shape)

    for feature in feature_list:
        i, j = feature.ravel()
        i, j = int(i), int(j)

        Ix = fx[i-w:i+w+1, j-w:j+w+1].flatten()
        Iy = fy[i-w:i+w+1, j-w:j+w+1].flatten()
        It = ft[i-w:i+w+1, j-w:j+w+1].flatten()

        b = -It  # Negate It to move it to the right-hand side of the equation
        A = np.vstack((Ix, Iy)).T

        # Add a regularization term to A.T @ A for numerical stability
        nu = 0.001
        U = np.linalg.pinv(A.T @ A + nu * np.eye(2)) @ A.T @ b

        u[i, j] = U[0]
        v[i, j] = U[1]

    return (u, v)


# Main function
- If part1 and part2 were filled properly, the 'output.avi' will be generated!
- For google colab, as cv2.imshow() is not provided, so please use cv2_imshow (google.colab.patches) instead  

In [7]:
if(isColab): cap = cv2.VideoCapture('/content/drive/MyDrive/optical_flow/slow.mp4')
else: cap = cv2.VideoCapture("C:/Users/labinno/Desktop/cv_project/slow.mp4")

# Take first frame and find corners in it
ret, old_frame = cap.read()

# Width and height of the file to save
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# 'output.mp4' will be generated!
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (int(width), int(height)))

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Shi Tomasi parameter
max_corners = 100
min_quality = 0.3
blocksize = 7
p0 = goodFeaturesToTrack(old_frame, max_corners, min_quality, blocksize)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, current_frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    U, V = optical_flow(old_gray, frame_gray, 15, 0.03)

    for i in range(current_frame.shape[0]):
        for j in range(current_frame.shape[1]):
            u, v = U[i][j], V[i][j]
            if u and v:
                mask = cv2.line(mask, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), 2)
                frame = cv2.arrowedLine(current_frame, (j, i), (int(round(j + u)), int(round(i + v))), (0, 255, 0), thickness=2)
                current_frame = cv2.add(current_frame, mask)

    # Display the frame with optical flow vectors
    if(isColab): cv2_imshow(current_frame)  # For Google Colab
    else: cv2.imshow('Lucas-Kanade Optical Flow', current_frame)
    
    out.write(current_frame)
    # Break the loop if 'Esc' key is pressed
    if cv2.waitKey(30) == 27:
        break

    # Set the current frame as the previous frame for the next iteration
    old_gray = frame_gray

# Release the video capture object
cap.release()
out.release()

# Close the plot window when done
plt.close()
cv2.destroyAllWindows()
